In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as nnf
from torchsummary import summary
import torchvision
import model.c2d as c2d
from torch.utils.data import Dataset
from torchnet.dataset.splitdataset import SplitDataset
import random
import os
import numpy as np
import glob

import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import math
import itertools
import sys
from torchvision.utils import save_image, make_grid
from torchvision.models import vgg19

from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import math
import cv2
from datetime import datetime
import time
import pandas as pd



In [3]:
# Data Loader

def vid_dataloader(vid_folder, audio_file, vid_fps, duration,batch_size):
    df = pd.read_csv(audio_file,header=None)
    vid_files = []
    for vid_file in os.listdir(vid_folder):
        vid_time_stamp = int(time.mktime(datetime.strptime(vid_file[0:-4], '%Y-%m-%d_%H-%M-%S').timetuple()))
        vid_obj = cv2.VideoCapture(os.path.join(vid_folder,vid_file))    
        vid_files.append([vid_obj,vid_obj.get(cv2.CAP_PROP_FRAME_COUNT),vid_time_stamp])

    print('files read')

    while(1):
        indices = np.random.randint(0,len(vid_files),size=batch_size)
        x_batch = []
        y_batch = []
        for i in indices:
            vid_tuple = vid_files[i]
            last_second = int(vid_tuple[1]/vid_fps) - duration
            if last_second < 0:
                continue

            # print(int(vid_tuple[1]/vid_fps))
            # print(last_second)
            vid_start_second =  np.random.randint(0,last_second+1)
            audio_start_tstamp = vid_tuple[2] + vid_start_second + math.ceil(duration/2)
            video_snippet_start_index = vid_start_second*vid_fps
            video_snippet_end_index = video_snippet_start_index + (duration*vid_fps)
            vid_tuple[0].set(cv2.CAP_PROP_POS_FRAMES,video_snippet_start_index)

            y = df[df[0] == audio_start_tstamp].drop(columns=[1]).to_numpy()
            if y.shape[0] < 1:
                continue

            frames = []
            for j in range(video_snippet_start_index,video_snippet_end_index):
                
                retval,frame = vid_tuple[0].read()
                if not retval:                    
                    print(retval,video_snippet_start_index,j,vid_tuple[1],vid_start_second,last_second)
                frames.append(frame)
            x_batch.append(np.concatenate(frames,axis=2))
            y_batch.append(y[0])
        y_batch = np.array(y_batch).astype(np.float32)
        x_batch = np.array(x_batch).astype(np.float32)
        width = x_batch.shape[2]
        x_batch = np.concatenate([x_batch[...,:int(width/2),:],x_batch[...,int(width/2):,:]],axis=3)
        # yield [np.array(x_batch),np.array(y_batch)]
        yield [torch.transpose(torch.tensor(x_batch),1,3),torch.tensor(y_batch)]



duration = 5
vid_fps = 5
batch_size = 64
vid_folder = 'G:/.shortcut-targets-by-id/1VaRYG8M-m7nfxKooARU6qTiHpVPhNHuV/out'
audio_file = 'G:/Shared drives/UF-AI-Catalyst/UF AI Code/test_data/1649855468-1649880078.csv'

loader = vid_dataloader(vid_folder,audio_file,vid_fps,duration,batch_size)





In [4]:
for data in loader:
    print(data[0].shape, data[1].shape,data[0].dtype)

files read
torch.Size([64, 150, 240, 240]) torch.Size([64, 32]) torch.float32
torch.Size([64, 150, 240, 240]) torch.Size([64, 32]) torch.float32
torch.Size([63, 150, 240, 240]) torch.Size([63, 32]) torch.float32
torch.Size([64, 150, 240, 240]) torch.Size([64, 32]) torch.float32


KeyboardInterrupt: 

In [13]:
# index = 74
# vid_files[indices[index]]
# img = Image.fromarray(x_batch[index][0])
# img.show()
# print(vid_files[indices[index]],y_batch[index])

[<VideoCapture 00000252271AE2B0>, 1499.0, 1649878188] 1649878463


In [2]:

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(0.5, 0.5)])
batch_size  = 100 
dataset = torchvision.datasets.MNIST(root='\\data',download=True,transform=transform)
params = {'batch_size': batch_size,
          'shuffle': True}


loader = torch.utils.data.DataLoader(dataset,**params)

In [56]:
in_channels = duration*vid_fps*3*2
out_dimension = 32

res = c2d.Resnet(in_channels,out_dimension,[3,1,1])
# res.model.train()
criterion_loss = nn.MSELoss()
optimizer = torch.optim.Adam(res.model.parameters(), lr = 2e-4)

i = 0
while(1):
    i+=1
    for data in loader:

        x = data[0]
        y = data[1]

        res.model.zero_grad()

        y_pred = res.model(x)
        loss = criterion_loss(y_pred,y)
        print(loss.item())

        loss.backward()
        optimizer.step()

        if not i%10:
            print(loss.item())


files read
8.506520314157466e+16
8.506540071007027e+16


In [52]:
for data in loader:
    print(res(data[0]).shape)

files read
torch.Size([64, 32])


KeyboardInterrupt: 

In [14]:
import os
from operator import itemgetter, attrgetter


def get_files_in_interval(strt,end,path):
    file_list = []
    for file in os.listdir(path):
        file_time = int(time.mktime(datetime.strptime(file[0:-4], '%Y-%m-%d_%H-%M-%S').timetuple()))
        if file_time >= strt and file_time < end:
            file_list.append([file_time,file,os.path.join(path,file)])
    return file_list


path1 ='G:/Shared drives/UF-AI-Catalyst/UF AI Code/test_data/bosch'
path2 = 'G:/Shared drives/UF-AI-Catalyst/UF AI Code/test_data/iteris'  
strt_time = 1649999413
end_time = 1650137713
fps = 5


file_list1 = get_files_in_interval(strt_time,end_time,path1)
file_list2 = get_files_in_interval(strt_time,end_time,path2)

print(len(file_list1))
print(len(file_list2))
# while(len(file_list2) and len(file_list1)):
#     if file_list1[0][0] > file_list2[0][0]:
#         ref_list = file_list1
#         scroll_list = file_list2
#     else:
#         ref_list = file_list2
#         scroll_list = file_list1

#     while(ref_list[0][0] >= scroll_list[0][0] and ref_list[0][0] < scroll_list[1][0]):

while(len(file_list2) and len(file_list1)):
    if file_list1[0][0] > file_list2[0][0]:
        efile = file_list2[0]
        lfile = file_list1[0]
    else:        
        efile = file_list1[0]
        lfile = file_list2[0]


    evid = cv2.VideoCapture(efile[2])
    fps = evid.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
    frame_count = int(evid.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps
    if (efile[0] + duration) > lfile[0]:
        lvid = cv2.VideoCapture(efile[2])
        evid.set(cv2.CV_CAP_PROP_POS_FRAMES,fps*(lvid[0]-evid[0]))
        




IndentationError: expected an indented block (Temp/ipykernel_15916/3693431966.py, line 2)

In [39]:
torch.ones((62,150,240,240)).dtype

torch.float32